In [2]:
pip install kaggle numpy pandas librosa torchaudio torch torchvision scikit-learn tqdm matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


# Directories clean-up

In [3]:
import os
import shutil

# Directory to clean
output_dir = "/kaggle/working/preprocessed_t2_test_chunks"

# Check if the directory exists and delete its contents
if os.path.exists(output_dir):
    print(f"Clearing existing files in {output_dir}...")
    shutil.rmtree(output_dir)
    print(f"Deleted {output_dir} and all its contents.")
else:
    print(f"Directory {output_dir} does not exist yet.")

# Recreate the directory
os.makedirs(output_dir, exist_ok=True)
print(f"Recreated empty directory: {output_dir}")

# Verification
files = os.listdir(output_dir)
print(f"Files in {output_dir} after clearing: {files}")

Directory /kaggle/working/preprocessed_t2_test_chunks does not exist yet.
Recreated empty directory: /kaggle/working/preprocessed_t2_test_chunks
Files in /kaggle/working/preprocessed_t2_test_chunks after clearing: []


# Data Preprocessing: MFCC, LFCC, Chroma-STFT

In [6]:
import os
import numpy as np
import torch
import torchaudio
from torchaudio.transforms import MFCC, LFCC, Spectrogram
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}, GPU: {torch.cuda.get_device_name(0)}")

# Paths
DATASET_PATHS = {
    "fake_or_real": "/kaggle/input/the-fake-or-real-dataset/",
    "wild_deepfake": "/kaggle/input/in-the-wild-audio-deepfake/"
}

# Function to validate if a .wav file can be loaded
def validate_wav_file(file_path):
    try:
        waveform, sr = torchaudio.load(file_path)
        if waveform.size(1) == 0:
            print(f"Skipping empty file: {file_path}")
            return False
        return True
    except Exception as e:
        print(f"Skipping invalid file: {file_path}, Error: {e}")
        return False

# Function to collect audio files and assign labels
def get_audio_files_and_labels():
    audio_files = []
    labels = []
    counts = {"fake": 0, "real": 0}

    # Process In-the-Wild dataset
    wild_path = DATASET_PATHS["wild_deepfake"]
    wild_real_path = os.path.join(wild_path, "release_in_the_wild", "real")
    wild_fake_path = os.path.join(wild_path, "release_in_the_wild", "fake")

    for file in os.listdir(wild_real_path):
        if file.endswith(".wav"):
            file_path = os.path.join(wild_real_path, file)
            if validate_wav_file(file_path):
                audio_files.append(file_path)
                labels.append(0)  # Real
                counts["real"] += 1

    for file in os.listdir(wild_fake_path):
        if file.endswith(".wav"):
            file_path = os.path.join(wild_fake_path, file)
            if validate_wav_file(file_path):
                audio_files.append(file_path)
                labels.append(1)  # Fake
                counts["fake"] += 1

    print(f"In-the-Wild Counts: Real: {counts['real']}, Fake: {counts['fake']}")

    # Process Fake-or-Real dataset
    for_counts = {"fake": 0, "real": 0}
    for_base_path = DATASET_PATHS["fake_or_real"]
    for_folders = ["for-2sec", "for-norm", "for-original", "for-rerec"]
    for_subfolders = {
        "for-2sec": "for-2seconds",
        "for-norm": "for-norm",
        "for-original": "for-original",
        "for-rerec": "for-rerecorded"
    }
    splits = ["training", "validation", "testing"]

    for folder in for_folders:
        subfolder = for_subfolders[folder]
        for split in splits:
            real_path = os.path.join(for_base_path, folder, subfolder, split, "real")
            if os.path.exists(real_path):
                for file in os.listdir(real_path):
                    if file.endswith(".wav"):
                        file_path = os.path.join(real_path, file)
                        if validate_wav_file(file_path):
                            audio_files.append(file_path)
                            labels.append(0)  # Real
                            for_counts["real"] += 1

            fake_path = os.path.join(for_base_path, folder, subfolder, split, "fake")
            if os.path.exists(fake_path):
                for file in os.listdir(fake_path):
                    if file.endswith(".wav"):
                        file_path = os.path.join(fake_path, file)
                        if validate_wav_file(file_path):
                            audio_files.append(file_path)
                            labels.append(1)  # Fake
                            for_counts["fake"] += 1

    print(f"Fake-or-Real Counts: Real: {for_counts['real']}, Fake: {for_counts['fake']}")
    total_counts = {"real": counts['real'] + for_counts['real'], "fake": counts['fake'] + for_counts['fake']}
    print(f"Total Counts: Real: {total_counts['real']}, Fake: {total_counts['fake']}")
    return audio_files, labels

# Collect files and labels
all_files, all_labels = get_audio_files_and_labels()
print(f"\nTotal files before processing: {len(all_files)}")
print(f"Label Distribution before processing: Real (0): {sum(1 for label in all_labels if label == 0)} ({sum(1 for label in all_labels if label == 0)/len(all_files)*100:.2f}%), Fake (1): {sum(1 for label in all_labels if label == 1)} ({sum(1 for label in all_labels if label == 1)/len(all_files)*100:.2f}%)")

# Custom Chroma-STFT implementation using torchaudio
def compute_chroma_stft(waveforms, sample_rate=16000, n_fft=2048, hop_length=512, n_chroma=12):
    # Compute spectrogram
    spectrogram_transform = Spectrogram(
        n_fft=n_fft,
        hop_length=hop_length,
        power=2.0
    ).to(device)
    spec = spectrogram_transform(waveforms)  # (batch, channels, freq, time)
    
    # Squeeze the channel dimension (since mono audio)
    spec = spec.squeeze(1)  # (batch, freq, time)
    
    # Create chroma filter bank
    freqs = torch.linspace(0, sample_rate / 2, steps=spec.shape[1]).to(device)  # Frequency bins
    chroma_freqs = torch.tensor([31.25 * (2 ** (i / 12)) for i in range(12 * 4)], device=device)  # MIDI 21 to 68 (A0 to C5)
    chroma_bins = torch.zeros((n_chroma, spec.shape[1]), device=device)
    
    for i in range(n_chroma):
        center = chroma_freqs[i::12]  # All octaves of this note
        for cf in center:
            mask = (freqs >= cf / 1.06) & (freqs <= cf * 1.06)  # Approximate triangular filter
            chroma_bins[i] += mask.float()
    
    # Normalize filter bank
    chroma_bins /= chroma_bins.sum(dim=1, keepdim=True).clamp(min=1e-10)
    
    # Apply filter bank to spectrogram
    chroma = torch.einsum('cf,bft->bct', chroma_bins, spec)  # (batch, n_chroma, time)
    return chroma

# Feature Extraction
def extract_features_batch(file_paths, labels, batch_size=64, max_length=16000):
    mfcc_results = []
    lfcc_results = []
    chroma_results = []
    valid_labels = []
    
    # Define feature extractors
    mfcc_transform = MFCC(
        sample_rate=16000,
        n_mfcc=40,
        melkwargs={"n_fft": 2048, "hop_length": 512, "n_mels": 128}
    ).to(device)
    
    lfcc_transform = LFCC(
        sample_rate=16000,
        n_lfcc=40,
        f_min=0,
        f_max=8000,
        n_filter=128,
        speckwargs={"n_fft": 2048, "hop_length": 512}
    ).to(device)
    
    total_batches = (len(file_paths) + batch_size - 1) // batch_size
    log_interval = max(1, total_batches // 100)  # Logging every 1% progress to check for errors
    
    for i in tqdm(range(0, len(file_paths), batch_size), desc="Processing Batches", total=total_batches):
        if i % (log_interval * batch_size) == 0:
            print(f"Processed {i // batch_size}/{total_batches} batches ({(i / len(file_paths)) * 100:.1f}%)")
            
        batch_files = file_paths[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        waveforms = []
        valid_indices = []
        
        # Load and preprocess waveforms
        for idx, (file_path, label) in enumerate(zip(batch_files, batch_labels)):
            try:
                waveform, sr = torchaudio.load(file_path)
                # Convert to mono by averaging channels
                if waveform.shape[0] > 1:
                    waveform = waveform.mean(dim=0, keepdim=True)  # (channels, samples) -> (1, samples)
                if sr != 16000:
                    waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
                
                if waveform.size(1) > max_length:
                    waveform = waveform[:, :max_length]
                elif waveform.size(1) < max_length:
                    pad_size = max_length - waveform.size(1)
                    waveform = torch.nn.functional.pad(waveform, (0, pad_size))
                
                waveforms.append(waveform)
                valid_indices.append(idx)
            except Exception as e:
                continue
        
        if not waveforms:
            continue
        
        try:
            waveforms = torch.nn.utils.rnn.pad_sequence(waveforms, batch_first=True).to(device)
            
            # Extract MFCC
            mfccs = mfcc_transform(waveforms)  # (batch, n_mfcc, time)
            
            # Extract LFCC
            lfccs = lfcc_transform(waveforms)  # (batch, n_lfcc, time)
            
            # Extract Chroma-STFT
            chromas = compute_chroma_stft(
                waveforms,
                sample_rate=16000,
                n_fft=2048,
                hop_length=512,
                n_chroma=12
            )  # (batch, n_chroma, time)
            
            # Standardize time dimension (trim/pad to 32 frames for consistency)
            target_frames = 32
            for idx in valid_indices:
                # MFCC
                mfcc = mfccs[idx]  # (n_mfcc, time)
                if mfcc.shape[1] > target_frames:
                    mfcc = mfcc[:, :target_frames]
                elif mfcc.shape[1] < target_frames:
                    mfcc = torch.nn.functional.pad(mfcc, (0, target_frames - mfcc.shape[1]))
                mfcc_results.append(mfcc.cpu().numpy())
                
                # LFCC
                lfcc = lfccs[idx]  # (n_lfcc, time)
                if lfcc.shape[1] > target_frames:
                    lfcc = lfcc[:, :target_frames]
                elif lfcc.shape[1] < target_frames:
                    lfcc = torch.nn.functional.pad(lfcc, (0, target_frames - lfcc.shape[1]))
                lfcc_results.append(lfcc.cpu().numpy())
                
                # Chroma-STFT
                chroma = chromas[idx]  # (n_chroma, time)
                if chroma.shape[1] > target_frames:
                    chroma = chroma[:, :target_frames]
                elif chroma.shape[1] < target_frames:
                    chroma = torch.nn.functional.pad(chroma, (0, target_frames - chroma.shape[1]))
                chroma_results.append(chroma.cpu().numpy())
                
                valid_labels.append(batch_labels[idx])
        
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            continue
    
    return mfcc_results, lfcc_results, chroma_results, valid_labels

# Process files in batches
mfcc_features, lfcc_features, chroma_features, y = extract_features_batch(all_files, all_labels, batch_size=64, max_length=16000)

# Convert to numpy arrays
mfcc_features = np.array(mfcc_features)  # (num_samples, n_mfcc, time)
lfcc_features = np.array(lfcc_features)  # (num_samples, n_lfcc, time)
chroma_features = np.array(chroma_features)  # (num_samples, n_chroma, time)
y = np.array(y)

# Validate alignment
if not (len(mfcc_features) == len(lfcc_features) == len(chroma_features) == len(y)):
    raise ValueError(f"Mismatch between features and labels: "
                     f"MFCC: {len(mfcc_features)}, LFCC: {len(lfcc_features)}, "
                     f"Chroma: {len(chroma_features)}, y: {len(y)}")

# Verify total samples
total_samples = len(y)
print(f"Total samples after feature extraction: {total_samples}")
if total_samples != 173128:
    print(f"Warning: Expected 173,128 samples, but got {total_samples} samples. Some files may have been skipped due to errors.")

# Save in chunks (268 chunks of 646 samples each)
num_chunks = 268
chunk_size = total_samples // num_chunks  # Should be 646
total_samples_used = num_chunks * chunk_size
print(f"Total samples: {total_samples}, Number of chunks: {num_chunks}, Chunk size: {chunk_size}")
print(f"Total samples used: {total_samples_used}")

# Adjustments if total_samples is not exactly divisible by chunk_size
if total_samples != total_samples_used:  # Shouldnt be, dont waste data
    print(f"Warning: {total_samples - total_samples_used} samples will be dropped due to chunking.")

output_dir = "/kaggle/working/preprocessed_chunks_268"
os.makedirs(output_dir, exist_ok=True)

# Clear existing chunks
for f in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, f))

# Save the data in chunks
for i in range(0, total_samples_used, chunk_size):
    chunk_idx = i // chunk_size
    mfcc_chunk = mfcc_features[i:i + chunk_size]  # (chunk_size, n_mfcc, time)
    lfcc_chunk = lfcc_features[i:i + chunk_size]  # (chunk_size, n_lfcc, time)
    chroma_chunk = chroma_features[i:i + chunk_size]  # (chunk_size, n_chroma, time)
    y_chunk = y[i:i + chunk_size]  # (chunk_size,)
    
    if not (len(mfcc_chunk) == len(lfcc_chunk) == len(chroma_chunk) == len(y_chunk)):
        raise ValueError(f"Mismatch in chunk {chunk_idx}: "
                         f"MFCC: {len(mfcc_chunk)}, LFCC: {len(lfcc_chunk)}, "
                         f"Chroma: {len(chroma_chunk)}, y: {len(y_chunk)}")
    
    np.save(os.path.join(output_dir, f"mfcc_chunk_{chunk_idx}.npy"), mfcc_chunk)
    np.save(os.path.join(output_dir, f"lfcc_chunk_{chunk_idx}.npy"), lfcc_chunk)
    np.save(os.path.join(output_dir, f"chroma_chunk_{chunk_idx}.npy"), chroma_chunk)
    np.save(os.path.join(output_dir, f"y_chunk_{chunk_idx}.npy"), y_chunk)
    print(f"Saved chunk {chunk_idx}: "
          f"MFCC shape {mfcc_chunk.shape}, LFCC shape {lfcc_chunk.shape}, "
          f"Chroma shape {chroma_chunk.shape}, y shape {y_chunk.shape}")

print(f"Processed and saved {total_samples_used} audio samples in {output_dir}!")

# Verify saved files
print("\nVerifying all saved files...")
chunk_files = [f for f in os.listdir(output_dir) if f.endswith('.npy')]
print(f"Found {len(chunk_files)} .npy files in {output_dir}:")
for chunk_file in sorted(chunk_files):
    file_path = os.path.join(output_dir, chunk_file)
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"{chunk_file}: {size_mb:.2f} MB")

Using device: cuda, GPU: Tesla T4
In-the-Wild Counts: Real: 19963, Fake: 11816
Skipping invalid file: /kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real/file15440.wav_16k.wav_norm.wav_mono.wav_silence.wav, Error: Failed to decode audio.
Skipping invalid file: /kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real/file11064.wav_16k.wav_norm.wav_mono.wav_silence.wav, Error: Failed to decode audio.
Fake-or-Real Counts: Real: 84756, Fake: 56593
Total Counts: Real: 104719, Fake: 68409

Total files before processing: 173128
Label Distribution before processing: Real (0): 104719 (60.49%), Fake (1): 68409 (39.51%)


Processing Batches:   0%|          | 0/2706 [00:00<?, ?it/s]

Processed 0/2706 batches (0.0%)


Processing Batches:   1%|          | 27/2706 [00:07<11:10,  3.99it/s]

Processed 27/2706 batches (1.0%)


...

Processed 2673/2706 batches (98.8%)


Processing Batches: 100%|█████████▉| 2700/2706 [34:58<00:03,  1.73it/s]

Processed 2700/2706 batches (99.8%)


Processing Batches: 100%|██████████| 2706/2706 [35:01<00:00,  1.29it/s]


Total samples after feature extraction: 173128
Total samples: 173128, Number of chunks: 268, Chunk size: 646
Total samples used: 173128
Saved chunk 0: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 1: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 2: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 3: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 4: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 5: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 6: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 12, 32), y shape (646,)
Saved chunk 7: MFCC shape (646, 1, 32, 32), LFCC sha

# Model Training and Testing - Datasets aggregated

In [8]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import psutil

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

epochs = 20  # Keep epochs with early stopping

# Dataset Class
class AudioDataset(Dataset):
    def __init__(self, chunk_dir, chunk_indices, dataset_type="train"):
        self.chunk_dir = chunk_dir
        self.dataset_type = dataset_type
        
        # Verify that the directory exists
        if not os.path.exists(chunk_dir):
            raise FileNotFoundError(f"Chunk directory {chunk_dir} does not exist.")
        
        # Look for MFCC chunk files with the correct names
        self.chunk_files = sorted([f for f in os.listdir(chunk_dir) if f.startswith("mfcc_chunk_") and f.endswith(".npy")])
        
        # Check if any chunk files were found; debugging for previous errors
        if not self.chunk_files:
            raise FileNotFoundError(f"No MFCC chunk files found in {chunk_dir}. Expected files like 'mfcc_chunk_0.npy'.")
        
        print(f"Found {len(self.chunk_files)} chunk files in {chunk_dir}. First few: {self.chunk_files[:5]}")
        
        self.chunk_size = 646  # Fixed chunk size from preprocessing
        self.num_chunks = len(self.chunk_files)
        self.total_samples = self.num_chunks * self.chunk_size
        
        # Filter indices to ensure they are within bounds
        self.indices = [idx for idx in chunk_indices if idx < self.total_samples]
        
        # Check if indices are empty
        if not self.indices:
            raise ValueError(f"No valid indices for {self.dataset_type} dataset. "
                            f"Total samples: {self.total_samples}, but chunk_indices range is {min(chunk_indices)} to {max(chunk_indices)}.")
        
        print(f"Dataset ({self.dataset_type}) initialized with {len(self.indices)} samples. "
              f"Index range: {self.indices[0]} to {self.indices[-1]}")
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        if idx >= len(self.indices):
            raise IndexError(f"Index {idx} is out of bounds for dataset with length {len(self.indices)}")
        
        # Map global index to chunk and local index
        global_idx = self.indices[idx]
        chunk_idx = global_idx // self.chunk_size
        local_idx = global_idx % self.chunk_size
        
        # Load the chunks with the correct file names
        mfcc_chunk = np.load(os.path.join(self.chunk_dir, f"mfcc_chunk_{chunk_idx}.npy"), mmap_mode='r')
        lfcc_chunk = np.load(os.path.join(self.chunk_dir, f"lfcc_chunk_{chunk_idx}.npy"), mmap_mode='r')
        chroma_chunk = np.load(os.path.join(self.chunk_dir, f"chroma_chunk_{chunk_idx}.npy"), mmap_mode='r')
        y_chunk = np.load(os.path.join(self.chunk_dir, f"y_chunk_{chunk_idx}.npy"), mmap_mode='r')
        
        # Extract the sample
        mfcc = torch.FloatTensor(mfcc_chunk[local_idx])  # Shape: (1, 32, 32)
        lfcc = torch.FloatTensor(lfcc_chunk[local_idx])  # Shape: (1, 32, 32)
        chroma = torch.FloatTensor(chroma_chunk[local_idx]).unsqueeze(0)  # Shape: (12, 32) -> (1, 12, 32)
        y = torch.LongTensor([y_chunk[local_idx]])[0]  # Single label (0 or 1)
        
        # Normalize features (zero mean, unit variance)
        mfcc = (mfcc - mfcc.mean()) / (mfcc.std() + 1e-8)
        lfcc = (lfcc - lfcc.mean()) / (lfcc.std() + 1e-8)
        chroma = (chroma - chroma.mean()) / (chroma.std() + 1e-8)
        
        # Add noise to features during training for data augmentation
        if self.dataset_type == "train":
            noise_factor = 0.1
            mfcc += torch.randn_like(mfcc) * noise_factor
            lfcc += torch.randn_like(lfcc) * noise_factor
            chroma += torch.randn_like(chroma) * noise_factor
        
        return mfcc, lfcc, chroma, y

# MFAAN Model with 1D Convolutions
class MFAAN(nn.Module):
    def __init__(self):
        super(MFAAN, self).__init__()
        # MFCC Path: Input shape (batch, 32, 32)
        self.path_mfcc = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),  # Reduced to 16
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),  # (batch, 16, 16)
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),  # (batch, 32, 16)
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)  # (batch, 32, 8)
        )
        
        # LFCC Path: Input shape (batch, 32, 32)
        self.path_lfcc = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),  # Reduced to 16
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),  # (batch, 16, 16)
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),  # (batch, 32, 16)
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)  # (batch, 32, 8)
        )
        
        # Chroma Path: Input shape (batch, 12, 32)
        self.path_chroma = nn.Sequential(
            nn.Conv1d(in_channels=12, out_channels=16, kernel_size=3, stride=1, padding=1),  # Reduced to 16
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),  # (batch, 16, 16)
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),  # (batch, 32, 16)
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)  # (batch, 32, 8)
        )
        
        # Fully connected layers after fusion
        # Each path outputs (batch, 32, 8), so after flattening and concatenation: 32 * 8 * 3 = 768
        self.fc1 = nn.Linear(32 * 8 * 3, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 2)  # Binary classification (real vs fake)
    
    def forward(self, mfcc, lfcc, chroma):
        mfcc = mfcc.squeeze(1)  # (batch, 1, 32, 32) -> (batch, 32, 32)
        lfcc = lfcc.squeeze(1)  # (batch, 1, 32, 32) -> (batch, 32, 32)
        chroma = chroma.squeeze(1)  # (batch, 1, 12, 32) -> (batch, 12, 32)
        
        # Process each feature through their path
        mfcc_out = self.path_mfcc(mfcc)  # (batch, 32, 8)
        lfcc_out = self.path_lfcc(lfcc)  # (batch, 32, 8)
        chroma_out = self.path_chroma(chroma)  # (batch, 32, 8)
        
        # Flatten and concatenate - finally
        mfcc_flat = mfcc_out.view(mfcc_out.size(0), -1)  # (batch, 32 * 8)
        lfcc_flat = lfcc_out.view(lfcc_out.size(0), -1)  # (batch, 32 * 8)
        chroma_flat = chroma_out.view(chroma_out.size(0), -1)  # (batch, 32 * 8)
        fused = torch.cat((mfcc_flat, lfcc_flat, chroma_flat), dim=1)  # (batch, 32 * 8 * 3)
        
        # Pass through fully connected layers
        x = F.relu(self.fc1(fused))
        x = self.dropout(x)
        output = self.fc2(x)
        return output

# Function to compute Equal Error Rate (EER)
def compute_eer(labels, scores):
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.absolute(fnr - fpr))]
    eer = fpr[np.nanargmin(np.absolute(fnr - fpr))]
    return eer

chunk_dir = "/kaggle/working/preprocessed_chunks_268"
total_samples = 173128

# For stratified split
all_labels = []
for chunk_idx in range(268):
    y_chunk = np.load(os.path.join(chunk_dir, f"y_chunk_{chunk_idx}.npy"))
    all_labels.extend(y_chunk)
all_labels = np.array(all_labels)

indices = list(range(len(all_labels)))

# Stratified split: train (80%), val (10%), test (10%)
train_indices, temp_indices, train_labels, temp_labels = train_test_split(
    indices, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)
val_indices, test_indices, val_labels, test_labels = train_test_split(
    temp_indices, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

print(f"Train samples: {len(train_indices)}, Val samples: {len(val_indices)}, Test samples: {len(test_indices)}")
print("Train class distribution:", np.bincount(train_labels))
print("Val class distribution:", np.bincount(val_labels))
print("Test class distribution:", np.bincount(test_labels))

train_dataset = AudioDataset(chunk_dir, train_indices, dataset_type="train")
val_dataset = AudioDataset(chunk_dir, val_indices, dataset_type="val")
test_dataset = AudioDataset(chunk_dir, test_indices, dataset_type="test")

# Debug - print dataset lengths
print(f"Train dataset length: {len(train_dataset)}")
print(f"Val dataset length: {len(val_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0)

# Check for class imbalance and apply class weights
class_counts = np.bincount(all_labels)
class_weights = 1.0 / class_counts
class_weights = torch.FloatTensor(class_weights / class_weights.sum()).to(device)
print(f"Class weights: {class_weights}")

# Model, Optimizer, and Loss
model = MFAAN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-3)
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)

# Training loop - early stopping based on eer
def train_model(epochs=epochs, patience=5):
    best_val_eer = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_idx, (mfcc, lfcc, chroma, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(mfcc, lfcc, chroma)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_preds = []
        val_scores = []
        val_labels = []
        with torch.no_grad():
            for mfcc, lfcc, chroma, labels in val_loader:
                mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
                outputs = model(mfcc, lfcc, chroma)
                probs = F.softmax(outputs, dim=1)[:, 1]  # Probability of class 1 (fake)
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_scores.extend(probs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        val_eer = compute_eer(val_labels, val_scores)
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, "
              f"Validation Accuracy: {val_acc * 100:.2f}%, Validation EER: {val_eer * 100:.2f}%")
        
        # Test after first epoch - see how it is
        if epoch == 0:
            test_preds = []
            test_scores = []
            test_labels = []
            with torch.no_grad():
                for batch_idx, (mfcc, lfcc, chroma, labels) in enumerate(test_loader):
                    mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
                    outputs = model(mfcc, lfcc, chroma)
                    probs = F.softmax(outputs, dim=1)[:, 1]
                    _, predicted = torch.max(outputs, 1)
                    test_preds.extend(predicted.cpu().numpy())
                    test_scores.extend(probs.cpu().numpy())
                    test_labels.extend(labels.cpu().numpy())
            test_acc = accuracy_score(test_labels, test_preds)
            test_eer = compute_eer(test_labels, test_scores)
            print(f"Test Accuracy after Epoch 1: {test_acc * 100:.2f}%, Test EER: {test_eer * 100:.2f}%")
        
        # Early Stopping based on eer
        if val_eer < best_val_eer:
            best_val_eer = val_eer
            patience_counter = 0
            torch.save(model.state_dict(), "/kaggle/working/mfaan_best.pth")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break
        
        scheduler.step(val_eer)

train_model(epochs=epochs, patience=5)

process = psutil.Process()
mem_info = process.memory_info()
print(f"Memory usage after training: {mem_info.rss / 1024**2:.2f} MB")

# Final Test Evaluation with Per-Class Accuracy
model.load_state_dict(torch.load("/kaggle/working/mfaan_best.pth"))
model.eval()
test_preds = []
test_scores = []
test_labels = []
with torch.no_grad():
    for mfcc, lfcc, chroma, labels in test_loader:
        mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
        outputs = model(mfcc, lfcc, chroma)
        probs = F.softmax(outputs, dim=1)[:, 1]
        _, predicted = torch.max(outputs, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_scores.extend(probs.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_eer = compute_eer(test_labels, test_scores)

# Compute per-class accuracy - fake and real
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, test_preds)
per_class_acc = cm.diagonal() / cm.sum(axis=1)
print(f"\nFinal Test Accuracy: {test_acc * 100:.2f}%, Final Test EER: {test_eer * 100:.2f}%")
print(f"Per-class accuracy (real, fake): {per_class_acc * 100}")

Using device: cuda
Train samples: 138502, Val samples: 17313, Test samples: 17313
Train class distribution: [83775 54727]
Val class distribution: [10472  6841]
Test class distribution: [10472  6841]
Found 268 chunk files in /kaggle/working/preprocessed_chunks_268. First few: ['mfcc_chunk_0.npy', 'mfcc_chunk_1.npy', 'mfcc_chunk_10.npy', 'mfcc_chunk_100.npy', 'mfcc_chunk_101.npy']
Dataset (train) initialized with 138502 samples. Index range: 125083 to 68149
Found 268 chunk files in /kaggle/working/preprocessed_chunks_268. First few: ['mfcc_chunk_0.npy', 'mfcc_chunk_1.npy', 'mfcc_chunk_10.npy', 'mfcc_chunk_100.npy', 'mfcc_chunk_101.npy']
Dataset (val) initialized with 17313 samples. Index range: 112480 to 130832
Found 268 chunk files in /kaggle/working/preprocessed_chunks_268. First few: ['mfcc_chunk_0.npy', 'mfcc_chunk_1.npy', 'mfcc_chunk_10.npy', 'mfcc_chunk_100.npy', 'mfcc_chunk_101.npy']
Dataset (test) initialized with 17313 samples. Index range: 168466 to 60314
Train dataset length: 

Epoch 1/20: 100%|██████████| 4329/4329 [02:52<00:00, 25.17it/s]


Epoch 1/20, Loss: 0.2952, Validation Accuracy: 94.73%, Validation EER: 4.22%
Test Accuracy after Epoch 1: 94.70%, Test EER: 4.27%


Epoch 2/20: 100%|██████████| 4329/4329 [02:57<00:00, 24.44it/s]


Epoch 2/20, Loss: 0.1641, Validation Accuracy: 96.52%, Validation EER: 3.28%


...

Epoch 18/20, Loss: 0.0883, Validation Accuracy: 98.86%, Validation EER: 1.17%


Epoch 19/20: 100%|██████████| 4329/4329 [02:52<00:00, 25.04it/s]


Epoch 19/20, Loss: 0.0898, Validation Accuracy: 98.73%, Validation EER: 1.27%


Epoch 20/20: 100%|██████████| 4329/4329 [02:53<00:00, 24.89it/s]


Epoch 20/20, Loss: 0.0883, Validation Accuracy: 98.79%, Validation EER: 1.22%
Memory usage after training: 4603.22 MB

Final Test Accuracy: 98.99%, Final Test EER: 1.04%
Per-class accuracy (real, fake): [99.26470588 98.5674609 ]


In [ ]:
import numpy as np

#to check for calss imbalance - it exists - need to use stratified
train_labels = [train_dataset[i][-1].item() for i in range(len(train_dataset))]
val_labels = [val_dataset[i][-1].item() for i in range(len(val_dataset))]
test_labels = [test_dataset[i][-1].item() for i in range(len(test_dataset))]
print("Train class distribution:", np.bincount(train_labels))
print("Val class distribution:", np.bincount(val_labels))
print("Test class distribution:", np.bincount(test_labels))

# Multi-Lingual Audio Deepfake Detection Corpus (MLADDC):
Dataset with halftruths. 

Initially, testing FoR & InTheWild trained model on T2 - international languages - dataset.

In [9]:
import os
import numpy as np
import torch
import torchaudio
from torchaudio.transforms import MFCC, LFCC, Spectrogram
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}, GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# T2 dataset path
T2_PATH = "/kaggle/input/mladdc-t2/T2/"

# Function to validate if a .wav file can be loaded
def validate_wav_file(file_path):
    try:
        waveform, sr = torchaudio.load(file_path)
        if waveform.size(1) == 0:
            print(f"Skipping empty file: {file_path}")
            return False
        return True
    except Exception as e:
        print(f"Skipping invalid file: {file_path}, Error: {e}")
        return False

# Function to collect T2 test audio files and labels
def get_t2_audio_files_and_labels(data_split="test"):
    audio_files = []
    labels = []
    counts = {"real": 0, "fake": 0}

    real_path = os.path.join(T2_PATH, data_split, "real")
    fake_path = os.path.join(T2_PATH, data_split, "deepfake")

    if not os.path.exists(real_path):
        raise FileNotFoundError(f"Real directory not found at {real_path}")
    if not os.path.exists(fake_path):
        raise FileNotFoundError(f"Fake directory not found at {fake_path}")

    for file in os.listdir(real_path):
        if file.endswith(".wav"):
            file_path = os.path.join(real_path, file)
            if validate_wav_file(file_path):
                audio_files.append(file_path)
                labels.append(0)  # Real
                counts["real"] += 1

    for file in os.listdir(fake_path):
        if file.endswith(".wav"):
            file_path = os.path.join(fake_path, file)
            if validate_wav_file(file_path):
                audio_files.append(file_path)
                labels.append(1)  # Fake
                counts["fake"] += 1

    print(f"T2 {data_split} Counts: Real: {counts['real']}, Fake: {counts['fake']}")
    return audio_files, labels

# Custom Chroma-STFT implementation
def compute_chroma_stft(waveforms, sample_rate=16000, n_fft=2048, hop_length=512, n_chroma=12):
    spectrogram_transform = Spectrogram(n_fft=n_fft, hop_length=hop_length, power=2.0).to(device)
    spec = spectrogram_transform(waveforms)
    spec = spec.squeeze(1)
    freqs = torch.linspace(0, sample_rate / 2, steps=spec.shape[1]).to(device)
    chroma_freqs = torch.tensor([31.25 * (2 ** (i / 12)) for i in range(12 * 4)], device=device)
    chroma_bins = torch.zeros((n_chroma, spec.shape[1]), device=device)
    for i in range(n_chroma):
        center = chroma_freqs[i::12]
        for cf in center:
            mask = (freqs >= cf / 1.06) & (freqs <= cf * 1.06)
            chroma_bins[i] += mask.float()
    chroma_bins /= chroma_bins.sum(dim=1, keepdim=True).clamp(min=1e-10)
    chroma = torch.einsum('cf,bft->bct', chroma_bins, spec)
    return chroma

# Feature Extraction for T2 test set
def extract_t2_features_batch(file_paths, labels, batch_size=64, max_length=16000):
    mfcc_results = []
    lfcc_results = []
    chroma_results = []
    valid_labels = []

    # Use 32 coefficients for MFCC and LFCC
    mfcc_transform = MFCC(
        sample_rate=16000,
        n_mfcc=32,
        melkwargs={"n_fft": 2048, "hop_length": 512, "n_mels": 128}
    ).to(device)
    lfcc_transform = LFCC(
        sample_rate=16000,
        n_lfcc=32,
        f_min=0,
        f_max=8000,
        n_filter=128,
        speckwargs={"n_fft": 2048, "hop_length": 512}
    ).to(device)

    total_batches = (len(file_paths) + batch_size - 1) // batch_size
    log_interval = max(1, total_batches // 100)

    for i in tqdm(range(0, len(file_paths), batch_size), desc="Processing T2 Test Batches", total=total_batches):
        if i % (log_interval * batch_size) == 0:
            print(f"Processed {i // batch_size}/{total_batches} batches ({(i / len(file_paths)) * 100:.1f}%)")

        batch_files = file_paths[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        waveforms = []
        valid_indices = []

        for idx, (file_path, label) in enumerate(zip(batch_files, batch_labels)):
            try:
                waveform, sr = torchaudio.load(file_path)
                if waveform.shape[0] > 1:
                    waveform = waveform.mean(dim=0, keepdim=True)
                if sr != 16000:
                    waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
                if waveform.size(1) > max_length:
                    waveform = waveform[:, :max_length]
                elif waveform.size(1) < max_length:
                    pad_size = max_length - waveform.size(1)
                    waveform = torch.nn.functional.pad(waveform, (0, pad_size))
                waveforms.append(waveform)
                valid_indices.append(idx)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
                continue

        if not waveforms:
            continue

        waveforms = torch.nn.utils.rnn.pad_sequence(waveforms, batch_first=True).to(device)
        mfccs = mfcc_transform(waveforms)  # Expected (batch, 32, time)
        lfccs = lfcc_transform(waveforms)  # Expected (batch, 32, time)
        chromas = compute_chroma_stft(waveforms, sample_rate=16000, n_fft=2048, hop_length=512, n_chroma=12)  # Expected (batch, 12, time)

        target_frames = 32
        for idx in valid_indices:
            # Process MFCC
            mfcc = mfccs[idx]  # Expected (32, time)
            if len(mfcc.shape) == 3:  # If (1, 32, time), remove extra dim
                mfcc = mfcc.squeeze(0)
            if mfcc.shape[1] > target_frames:
                mfcc = mfcc[:, :target_frames]
            elif mfcc.shape[1] < target_frames:
                mfcc = torch.nn.functional.pad(mfcc, (0, target_frames - mfcc.shape[1]))
            if mfcc.shape != (32, 32):
                print(f"Warning: MFCC shape after resize at index {idx}: {mfcc.shape}")
                continue
            mfcc = mfcc.unsqueeze(0)  # (32, 32) -> (1, 32, 32)
            if mfcc.shape != (1, 32, 32):
                print(f"Warning: Final MFCC shape mismatch at index {idx}: {mfcc.shape}")
                continue
            mfcc_results.append(mfcc.cpu().numpy())

            # Process LFCC
            lfcc = lfccs[idx]
            if len(lfcc.shape) == 3:
                lfcc = lfcc.squeeze(0)
            if lfcc.shape[1] > target_frames:
                lfcc = lfcc[:, :target_frames]
            elif lfcc.shape[1] < target_frames:
                lfcc = torch.nn.functional.pad(lfcc, (0, target_frames - lfcc.shape[1]))
            if lfcc.shape != (32, 32):
                print(f"Warning: LFCC shape after resize at index {idx}: {lfcc.shape}")
                continue
            lfcc = lfcc.unsqueeze(0)  # (32, 32) -> (1, 32, 32)
            if lfcc.shape != (1, 32, 32):
                print(f"Warning: Final LFCC shape mismatch at index {idx}: {lfcc.shape}")
                continue
            lfcc_results.append(lfcc.cpu().numpy())

            # Process Chroma
            chroma = chromas[idx]
            if len(chroma.shape) == 3:
                chroma = chroma.squeeze(0)
            if chroma.shape[1] > target_frames:
                chroma = chroma[:, :target_frames]
            elif chroma.shape[1] < target_frames:
                chroma = torch.nn.functional.pad(chroma, (0, target_frames - chroma.shape[1]))
            if chroma.shape != (12, 32):
                print(f"Warning: Chroma shape after resize at index {idx}: {chroma.shape}")
                continue
            chroma = chroma.unsqueeze(0)  # (12, 32) -> (1, 12, 32)
            if chroma.shape != (1, 12, 32):
                print(f"Warning: Final Chroma shape mismatch at index {idx}: {chroma.shape}")
                continue
            chroma_results.append(chroma.cpu().numpy())

            valid_labels.append(batch_labels[idx])

    return mfcc_results, lfcc_results, chroma_results, valid_labels

# Preprocess T2 test files
t2_test_files, t2_test_labels = get_t2_audio_files_and_labels("test")
print(f"\nTotal T2 test files before processing: {len(t2_test_files)}")
print(f"Label Distribution: Real (0): {sum(1 for label in t2_test_labels if label == 0)} "
      f"({sum(1 for label in t2_test_labels if label == 0)/len(t2_test_files)*100:.2f}%), "
      f"Fake (1): {sum(1 for label in t2_test_labels if label == 1)} "
      f"({sum(1 for label in t2_test_labels if label == 1)/len(t2_test_files)*100:.2f}%)")

# Extract features
t2_test_mfcc_features, t2_test_lfcc_features, t2_test_chroma_features, t2_test_y = extract_t2_features_batch(
    t2_test_files, t2_test_labels, batch_size=64, max_length=16000
)

# Convert to numpy arrays
t2_test_mfcc_features = np.array(t2_test_mfcc_features)  # (num_samples, 1, 32, 32)
t2_test_lfcc_features = np.array(t2_test_lfcc_features)  # (num_samples, 1, 32, 32)
t2_test_chroma_features = np.array(t2_test_chroma_features)  # (num_samples, 1, 12, 32)
t2_test_y = np.array(t2_test_y)  # (num_samples,)

# Debugging - validate shapes 
print(f"MFCC features shape: {t2_test_mfcc_features.shape}")
print(f"LFCC features shape: {t2_test_lfcc_features.shape}")
print(f"Chroma features shape: {t2_test_chroma_features.shape}")
print(f"Labels shape: {t2_test_y.shape}")

# Validate alignment
if not (len(t2_test_mfcc_features) == len(t2_test_lfcc_features) == len(t2_test_chroma_features) == len(t2_test_y)):
    raise ValueError(f"Mismatch between T2 test features and labels: "
                     f"MFCC: {len(t2_test_mfcc_features)}, LFCC: {len(t2_test_lfcc_features)}, "
                     f"Chroma: {len(t2_test_chroma_features)}, y: {len(t2_test_y)}")

# Verify total samples
total_t2_test_samples = len(t2_test_y)
print(f"Total T2 test samples after feature extraction: {total_t2_test_samples}")

# Save T2 test chunks
output_dir_t2_test = "/kaggle/working/preprocessed_t2_test_chunks"
os.makedirs(output_dir_t2_test, exist_ok=True)

# Clear existing T2 test chunks
for f in os.listdir(output_dir_t2_test):
    os.remove(os.path.join(output_dir_t2_test, f))

# Use chunk size of 646 to match training data
chunk_size_t2_test = 646
num_chunks_t2_test = (total_t2_test_samples + chunk_size_t2_test - 1) // chunk_size_t2_test
total_t2_test_samples_used = num_chunks_t2_test * chunk_size_t2_test

print(f"Total T2 test samples: {total_t2_test_samples}, Number of T2 test chunks: {num_chunks_t2_test}, "
      f"T2 Test Chunk size: {chunk_size_t2_test}")
if total_t2_test_samples != total_t2_test_samples_used:
    print(f"Warning: {total_t2_test_samples_used - total_t2_test_samples} samples will be padded due to chunking.")

# Pad features and labels if needed
if total_t2_test_samples < total_t2_test_samples_used:
    pad_size = total_t2_test_samples_used - total_t2_test_samples
    mfcc_pad = np.zeros((pad_size, 1, 32, 32), dtype=np.float32)
    lfcc_pad = np.zeros((pad_size, 1, 32, 32), dtype=np.float32)
    chroma_pad = np.zeros((pad_size, 1, 12, 32), dtype=np.float32)
    y_pad = np.zeros(pad_size, dtype=np.int64)  # Pad with real labels (0)

    t2_test_mfcc_features = np.concatenate([t2_test_mfcc_features, mfcc_pad], axis=0)
    t2_test_lfcc_features = np.concatenate([t2_test_lfcc_features, lfcc_pad], axis=0)
    t2_test_chroma_features = np.concatenate([t2_test_chroma_features, chroma_pad], axis=0)
    t2_test_y = np.concatenate([t2_test_y, y_pad], axis=0)

# Validate padded shapes
print(f"Padded MFCC features shape: {t2_test_mfcc_features.shape}")
print(f"Padded LFCC features shape: {t2_test_lfcc_features.shape}")
print(f"Padded Chroma features shape: {t2_test_chroma_features.shape}")
print(f"Padded Labels shape: {t2_test_y.shape}")

# Save chunks
for i in range(0, total_t2_test_samples_used, chunk_size_t2_test):
    chunk_idx = i // chunk_size_t2_test
    t2_test_mfcc_chunk = t2_test_mfcc_features[i:i + chunk_size_t2_test]
    t2_test_lfcc_chunk = t2_test_lfcc_features[i:i + chunk_size_t2_test]
    t2_test_chroma_chunk = t2_test_chroma_features[i:i + chunk_size_t2_test]
    t2_test_y_chunk = t2_test_y[i:i + chunk_size_t2_test]

    if not (len(t2_test_mfcc_chunk) == len(t2_test_lfcc_chunk) == len(t2_test_chroma_chunk) == len(t2_test_y_chunk)):
        raise ValueError(f"Mismatch in T2 test chunk {chunk_idx}: "
                         f"MFCC: {len(t2_test_mfcc_chunk)}, LFCC: {len(t2_test_lfcc_chunk)}, "
                         f"Chroma: {len(t2_test_chroma_chunk)}, y: {len(t2_test_y_chunk)}")

    np.save(os.path.join(output_dir_t2_test, f"t2_test_mfcc_chunk_{chunk_idx}.npy"), t2_test_mfcc_chunk)
    np.save(os.path.join(output_dir_t2_test, f"t2_test_lfcc_chunk_{chunk_idx}.npy"), t2_test_lfcc_chunk)
    np.save(os.path.join(output_dir_t2_test, f"t2_test_chroma_chunk_{chunk_idx}.npy"), t2_test_chroma_chunk)
    np.save(os.path.join(output_dir_t2_test, f"t2_test_y_chunk_{chunk_idx}.npy"), t2_test_y_chunk)
    print(f"Saved T2 test chunk {chunk_idx}: "
          f"MFCC shape {t2_test_mfcc_chunk.shape}, LFCC shape {t2_test_lfcc_chunk.shape}, "
          f"Chroma shape {t2_test_chroma_chunk.shape}, y shape {t2_test_y_chunk.shape}")

print(f"Processed and saved {total_t2_test_samples_used} T2 test audio samples in {output_dir_t2_test}!")

Using device: cuda, GPU: Tesla T4
T2 test Counts: Real: 5600, Fake: 11200

Total T2 test files before processing: 16800
Label Distribution: Real (0): 5600 (33.33%), Fake (1): 11200 (66.67%)


Processing T2 Test Batches:   0%|          | 0/263 [00:00<?, ?it/s]

Processed 0/263 batches (0.0%)


Processing T2 Test Batches:   1%|          | 2/263 [00:01<03:24,  1.27it/s]

Processed 2/263 batches (0.8%)


...

Processed 260/263 batches (99.0%)


Processing T2 Test Batches: 100%|█████████▉| 262/263 [03:23<00:00,  1.35it/s]

Processed 262/263 batches (99.8%)


Processing T2 Test Batches: 100%|██████████| 263/263 [03:23<00:00,  1.29it/s]


MFCC features shape: (16800, 1, 32, 32)
LFCC features shape: (16800, 1, 32, 32)
Chroma features shape: (16800, 1, 12, 32)
Labels shape: (16800,)
Total T2 test samples after feature extraction: 16800
Total T2 test samples: 16800, Number of T2 test chunks: 27, T2 Test Chunk size: 646
Padded MFCC features shape: (17442, 1, 32, 32)
Padded LFCC features shape: (17442, 1, 32, 32)
Padded Chroma features shape: (17442, 1, 12, 32)
Padded Labels shape: (17442,)
Saved T2 test chunk 0: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 1, 12, 32), y shape (646,)
Saved T2 test chunk 1: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 1, 12, 32), y shape (646,)
Saved T2 test chunk 2: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 1, 12, 32), y shape (646,)
Saved T2 test chunk 3: MFCC shape (646, 1, 32, 32), LFCC shape (646, 1, 32, 32), Chroma shape (646, 1, 12, 32), y shape (646,)
Saved T2 test chunk 4: MFCC shape (6

## Preprocessing for Training and Validation folders

In [10]:

import os
import numpy as np
import torch
import torch.nn as nn
import torchaudio
from torchaudio.transforms import MFCC, LFCC, Spectrogram
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}, GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# T2 dataset path
T2_PATH = "/kaggle/input/mladdc-t2/T2/"

# Function to validate if a .wav file can be loaded
def validate_wav_file(file_path):
    try:
        waveform, sr = torchaudio.load(file_path)
        if waveform.size(1) == 0:
            print(f"Skipping empty file: {file_path}")
            return False
        return True
    except Exception as e:
        print(f"Skipping invalid file: {file_path}, Error: {e}")
        return False

# Function to collect T2 audio files and labels
def get_t2_audio_files_and_labels(data_split="train"):
    audio_files = []
    labels = []
    counts = {"real": 0, "fake": 0}

    real_path = os.path.join(T2_PATH, data_split, "real")
    fake_path = os.path.join(T2_PATH, data_split, "deepfake")

    if not os.path.exists(real_path):
        raise FileNotFoundError(f"Real directory not found at {real_path}")
    if not os.path.exists(fake_path):
        raise FileNotFoundError(f"Fake directory not found at {fake_path}")

    for file in os.listdir(real_path):
        if file.endswith(".wav"):
            file_path = os.path.join(real_path, file)
            if validate_wav_file(file_path):
                audio_files.append(file_path)
                labels.append(0)  # Real
                counts["real"] += 1

    for file in os.listdir(fake_path):
        if file.endswith(".wav"):
            file_path = os.path.join(fake_path, file)
            if validate_wav_file(file_path):
                audio_files.append(file_path)
                labels.append(1)  # Fake
                counts["fake"] += 1

    print(f"T2 {data_split} Counts: Real: {counts['real']}, Fake: {counts['fake']}")
    return audio_files, labels

# Custom Chroma-STFT implementation
def compute_chroma_stft(waveforms, sample_rate=16000, n_fft=2048, hop_length=512, n_chroma=12):
    spectrogram_transform = Spectrogram(n_fft=n_fft, hop_length=hop_length, power=2.0).to(device)
    spec = spectrogram_transform(waveforms)
    spec = spec.squeeze(1)
    freqs = torch.linspace(0, sample_rate / 2, steps=spec.shape[1]).to(device)
    chroma_freqs = torch.tensor([31.25 * (2 ** (i / 12)) for i in range(12 * 4)], device=device)
    chroma_bins = torch.zeros((n_chroma, spec.shape[1]), device=device)
    for i in range(n_chroma):
        center = chroma_freqs[i::12]
        for cf in center:
            mask = (freqs >= cf / 1.06) & (freqs <= cf * 1.06)
            chroma_bins[i] += mask.float()
    chroma_bins /= chroma_bins.sum(dim=1, keepdim=True).clamp(min=1e-10)
    chroma = torch.einsum('cf,bft->bct', chroma_bins, spec)
    return chroma

# Feature Extraction for T2 train/val set
def extract_t2_features_batch(file_paths, labels, batch_size=64, max_length=16000, data_split="train"):
    mfcc_results = []
    lfcc_results = []
    chroma_results = []
    valid_labels = []

    mfcc_transform = MFCC(
        sample_rate=16000,
        n_mfcc=32,
        melkwargs={"n_fft": 2048, "hop_length": 512, "n_mels": 128}
    ).to(device)
    lfcc_transform = LFCC(
        sample_rate=16000,
        n_lfcc=32,
        f_min=0,
        f_max=8000,
        n_filter=128,
        speckwargs={"n_fft": 2048, "hop_length": 512}
    ).to(device)

    total_batches = (len(file_paths) + batch_size - 1) // batch_size
    log_interval = max(1, total_batches // 100)

    for i in tqdm(range(0, len(file_paths), batch_size), desc=f"Processing T2 {data_split} Batches", total=total_batches):
        if i % (log_interval * batch_size) == 0:
            print(f"Processed {i // batch_size}/{total_batches} batches ({(i / len(file_paths)) * 100:.1f}%)")

        batch_files = file_paths[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        waveforms = []
        valid_indices = []

        for idx, (file_path, label) in enumerate(zip(batch_files, batch_labels)):
            try:
                waveform, sr = torchaudio.load(file_path)
                if waveform.shape[0] > 1:
                    waveform = waveform.mean(dim=0, keepdim=True)
                if sr != 16000:
                    waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
                if waveform.size(1) > max_length:
                    waveform = waveform[:, :max_length]
                elif waveform.size(1) < max_length:
                    pad_size = max_length - waveform.size(1)
                    waveform = torch.nn.functional.pad(waveform, (0, pad_size))
                waveforms.append(waveform)
                valid_indices.append(idx)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
                continue

        if not waveforms:
            continue

        waveforms = torch.nn.utils.rnn.pad_sequence(waveforms, batch_first=True).to(device)
        mfccs = mfcc_transform(waveforms)
        lfccs = lfcc_transform(waveforms)
        chromas = compute_chroma_stft(waveforms, sample_rate=16000, n_fft=2048, hop_length=512, n_chroma=12)

        # Debugging - recurring shape mismatch issues
        target_frames = 32
        for idx in valid_indices:
            mfcc = mfccs[idx]
            if len(mfcc.shape) == 3:
                mfcc = mfcc.squeeze(0)
            if mfcc.shape[1] > target_frames:
                mfcc = mfcc[:, :target_frames]
            elif mfcc.shape[1] < target_frames:
                mfcc = torch.nn.functional.pad(mfcc, (0, target_frames - mfcc.shape[1]))
            if mfcc.shape != (32, 32):
                print(f"Warning: MFCC shape after resize at index {idx}: {mfcc.shape}")
                continue
            mfcc = mfcc.unsqueeze(0)
            if mfcc.shape != (1, 32, 32):
                print(f"Warning: Final MFCC shape mismatch at index {idx}: {mfcc.shape}")
                continue
            mfcc_results.append(mfcc.cpu().numpy())

            lfcc = lfccs[idx]
            if len(lfcc.shape) == 3:
                lfcc = lfcc.squeeze(0)
            if lfcc.shape[1] > target_frames:
                lfcc = lfcc[:, :target_frames]
            elif lfcc.shape[1] < target_frames:
                lfcc = torch.nn.functional.pad(lfcc, (0, target_frames - lfcc.shape[1]))
            if lfcc.shape != (32, 32):
                print(f"Warning: LFCC shape after resize at index {idx}: {lfcc.shape}")
                continue
            lfcc = lfcc.unsqueeze(0)
            if lfcc.shape != (1, 32, 32):
                print(f"Warning: Final LFCC shape mismatch at index {idx}: {lfcc.shape}")
                continue
            lfcc_results.append(lfcc.cpu().numpy())

            chroma = chromas[idx]
            if len(chroma.shape) == 3:
                chroma = chroma.squeeze(0)
            if chroma.shape[1] > target_frames:
                chroma = chroma[:, :target_frames]
            elif chroma.shape[1] < target_frames:
                chroma = torch.nn.functional.pad(chroma, (0, target_frames - chroma.shape[1]))
            if chroma.shape != (12, 32):
                print(f"Warning: Chroma shape after resize at index {idx}: {chroma.shape}")
                continue
            chroma = chroma.unsqueeze(0)
            if chroma.shape != (1, 12, 32):
                print(f"Warning: Final Chroma shape mismatch at index {idx}: {chroma.shape}")
                continue
            chroma_results.append(chroma.cpu().numpy())

            valid_labels.append(batch_labels[idx])

    return mfcc_results, lfcc_results, chroma_results, valid_labels

# MFAAN Model
class MFAAN(nn.Module):
    def __init__(self):
        super(MFAAN, self).__init__()
        self.path_mfcc = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.path_lfcc = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.path_chroma = nn.Sequential(
            nn.Conv1d(in_channels=12, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(32 * 8 * 3, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, mfcc, lfcc, chroma):
        mfcc = mfcc.squeeze(1)
        lfcc = lfcc.squeeze(1)
        chroma = chroma.squeeze(1)
        mfcc_out = self.path_mfcc(mfcc)
        lfcc_out = self.path_lfcc(lfcc)
        chroma_out = self.path_chroma(chroma)
        mfcc_flat = mfcc_out.view(mfcc_out.size(0), -1)
        lfcc_flat = lfcc_out.view(lfcc_out.size(0), -1)
        chroma_flat = chroma_out.view(chroma_out.size(0), -1)
        fused = torch.cat((mfcc_flat, lfcc_flat, chroma_flat), dim=1)
        x = F.relu(self.fc1(fused))
        x = self.dropout(x)
        output = self.fc2(x)
        return output

# Calculate and print model size
model = MFAAN()
total_params = sum(p.numel() for p in model.parameters())
param_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
param_size_mb = param_size_bytes / (1024 ** 2)
print(f"\nMFAAN Model Size:")
print(f"Total Parameters: {total_params:,}")
print(f"Parameter Size: {param_size_mb:.2f} MB")

# Preprocess T2 train and val sets
for data_split in ["train", "val"]:
    print(f"\nProcessing T2 {data_split} split...")
    files, labels = get_t2_audio_files_and_labels(data_split)
    print(f"Total T2 {data_split} files: {len(files)}")
    print(f"Label Distribution: Real (0): {sum(1 for label in labels if label == 0)} "
          f"({sum(1 for label in labels if label == 0)/len(files)*100:.2f}%), "
          f"Fake (1): {sum(1 for label in labels if label == 1)} "
          f"({sum(1 for label in labels if label == 1)/len(files)*100:.2f}%)")

    mfcc_features, lfcc_features, chroma_features, labels = extract_t2_features_batch(
        files, labels, batch_size=64, max_length=16000, data_split=data_split
    )
    mfcc_features = np.array(mfcc_features)
    lfcc_features = np.array(lfcc_features)
    chroma_features = np.array(chroma_features)
    labels = np.array(labels)

    print(f"T2 {data_split} MFCC shape: {mfcc_features.shape}")
    print(f"T2 {data_split} LFCC shape: {lfcc_features.shape}")
    print(f"T2 {data_split} Chroma shape: {chroma_features.shape}")
    print(f"T2 {data_split} Labels shape: {labels.shape}")

    # Validate alignment
    if not (len(mfcc_features) == len(lfcc_features) == len(chroma_features) == len(labels)):
        raise ValueError(f"Mismatch in T2 {data_split} features and labels: "
                         f"MFCC: {len(mfcc_features)}, LFCC: {len(lfcc_features)}, "
                         f"Chroma: {len(chroma_features)}, Labels: {len(labels)}")

    # Save preprocessed data
    output_dir = f"/kaggle/working/preprocessed_t2_{data_split}"
    os.makedirs(output_dir, exist_ok=True)
    for f in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, f))  # Clear existing files
    np.save(os.path.join(output_dir, f"t2_{data_split}_mfcc.npy"), mfcc_features)
    np.save(os.path.join(output_dir, f"t2_{data_split}_lfcc.npy"), lfcc_features)
    np.save(os.path.join(output_dir, f"t2_{data_split}_chroma.npy"), chroma_features)
    np.save(os.path.join(output_dir, f"t2_{data_split}_labels.npy"), labels)
    print(f"Saved preprocessed T2 {data_split} data to {output_dir}")

Using device: cuda, GPU: Tesla T4

MFAAN Model Size:
Total Parameters: 206,066
Parameter Size: 0.79 MB

Processing T2 train split...
T2 train Counts: Real: 44800, Fake: 89600
Total T2 train files: 134400
Label Distribution: Real (0): 44800 (33.33%), Fake (1): 89600 (66.67%)


Processing T2 train Batches:   0%|          | 0/2100 [00:00<?, ?it/s]

Processed 0/2100 batches (0.0%)


Processing T2 train Batches:   1%|          | 21/2100 [00:24<40:15,  1.16s/it]

Processed 21/2100 batches (1.0%)


...

Processed 260/263 batches (99.0%)


Processing T2 val Batches: 100%|█████████▉| 262/263 [04:48<00:01,  1.06s/it]

Processed 262/263 batches (99.8%)


Processing T2 val Batches: 100%|██████████| 263/263 [04:49<00:00,  1.10s/it]


T2 val MFCC shape: (16800, 1, 32, 32)
T2 val LFCC shape: (16800, 1, 32, 32)
T2 val Chroma shape: (16800, 1, 12, 32)
T2 val Labels shape: (16800,)
Saved preprocessed T2 val data to /kaggle/working/preprocessed_t2_val


In [14]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}, GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# MFAAN Model
class MFAAN(nn.Module):
    def __init__(self):
        super(MFAAN, self).__init__()
        self.path_mfcc = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.path_lfcc = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.path_chroma = nn.Sequential(
            nn.Conv1d(in_channels=12, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(32 * 8 * 3, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, mfcc, lfcc, chroma):
        mfcc = mfcc.squeeze(1)
        lfcc = lfcc.squeeze(1)
        chroma = chroma.squeeze(1)
        mfcc_out = self.path_mfcc(mfcc)
        lfcc_out = self.path_lfcc(lfcc)
        chroma_out = self.path_chroma(chroma)
        mfcc_flat = mfcc_out.view(mfcc_out.size(0), -1)
        lfcc_flat = lfcc_out.view(lfcc_out.size(0), -1)
        chroma_flat = chroma_out.view(chroma_out.size(0), -1)
        fused = torch.cat((mfcc_flat, lfcc_flat, chroma_flat), dim=1)
        x = F.relu(self.fc1(fused))
        x = self.dropout(x)
        output = self.fc2(x)
        return output

model = MFAAN()
total_params = sum(p.numel() for p in model.parameters())
param_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
param_size_mb = param_size_bytes / (1024 ** 2)
print(f"\nMFAAN Model Size:")
print(f"Total Parameters: {total_params:,}")
print(f"Parameter Size: {param_size_mb:.2f} MB")

# Custom Dataset Class for Train/Val
class T2AudioDataset(Dataset):
    def __init__(self, mfcc_features, lfcc_features, chroma_features, labels):
        self.mfcc_features = mfcc_features
        self.lfcc_features = lfcc_features
        self.chroma_features = chroma_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        mfcc = torch.FloatTensor(self.mfcc_features[idx])
        lfcc = torch.FloatTensor(self.lfcc_features[idx])
        chroma = torch.FloatTensor(self.chroma_features[idx])
        label = torch.LongTensor([self.labels[idx]])[0]

        mfcc = (mfcc - mfcc.mean()) / (mfcc.std() + 1e-8)
        lfcc = (lfcc - lfcc.mean()) / (lfcc.std() + 1e-8)
        chroma = (chroma - chroma.mean()) / (chroma.std() + 1e-8)

        return mfcc, lfcc, chroma, label

# Custom Dataset Class for Test
class AudioDataset(Dataset):
    def __init__(self, chunk_dir, chunk_indices, dataset_type="test"):
        self.chunk_dir = chunk_dir
        self.dataset_type = dataset_type
        if not os.path.exists(chunk_dir):
            raise FileNotFoundError(f"Chunk directory {chunk_dir} does not exist.")
        
        self.chunk_files = sorted([f for f in os.listdir(chunk_dir) if f.startswith("t2_test_mfcc_chunk_") and f.endswith(".npy")])
        if not self.chunk_files:
            raise FileNotFoundError(f"No MFCC chunk files found in {chunk_dir}.")
        
        print(f"Found {len(self.chunk_files)} chunk files in {chunk_dir}. First few: {self.chunk_files[:5]}")
        
        self.chunk_size = 646
        self.num_chunks = len(self.chunk_files)
        self.total_samples = self.num_chunks * self.chunk_size
        self.indices = [idx for idx in chunk_indices if idx < self.total_samples]
        
        if not self.indices:
            raise ValueError(f"No valid indices for {self.dataset_type} dataset. Total samples: {self.total_samples}.")
        
        print(f"Dataset ({self.dataset_type}) initialized with {len(self.indices)} samples.")

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        if idx >= len(self.indices):
            raise IndexError(f"Index {idx} is out of bounds for dataset with length {len(self.indices)}")
        
        global_idx = self.indices[idx]
        chunk_idx = global_idx // self.chunk_size
        local_idx = global_idx % self.chunk_size
        
        mfcc_chunk = np.load(os.path.join(self.chunk_dir, f"t2_test_mfcc_chunk_{chunk_idx}.npy"), mmap_mode='r')
        lfcc_chunk = np.load(os.path.join(self.chunk_dir, f"t2_test_lfcc_chunk_{chunk_idx}.npy"), mmap_mode='r')
        chroma_chunk = np.load(os.path.join(self.chunk_dir, f"t2_test_chroma_chunk_{chunk_idx}.npy"), mmap_mode='r')
        y_chunk = np.load(os.path.join(self.chunk_dir, f"t2_test_y_chunk_{chunk_idx}.npy"), mmap_mode='r')
        
        mfcc = torch.FloatTensor(mfcc_chunk[local_idx])
        lfcc = torch.FloatTensor(lfcc_chunk[local_idx])
        chroma = torch.FloatTensor(chroma_chunk[local_idx])
        y = torch.LongTensor([y_chunk[local_idx]])[0]
        
        mfcc = (mfcc - mfcc.mean()) / (mfcc.std() + 1e-8)
        lfcc = (lfcc - lfcc.mean()) / (lfcc.std() + 1e-8)
        chroma = (chroma - chroma.mean()) / (chroma.std() + 1e-8)
        
        return mfcc, lfcc, chroma, y

# Compute eer
def compute_eer(labels, scores):
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    eer = fpr[np.nanargmin(np.absolute(fnr - fpr))]
    return eer

# Load preprocessed train and val data
try:
    train_dataset = T2AudioDataset(
        np.load("/kaggle/working/preprocessed_t2_train/t2_train_mfcc.npy"),
        np.load("/kaggle/working/preprocessed_t2_train/t2_train_lfcc.npy"),
        np.load("/kaggle/working/preprocessed_t2_train/t2_train_chroma.npy"),
        np.load("/kaggle/working/preprocessed_t2_train/t2_train_labels.npy")
    )
    val_dataset = T2AudioDataset(
        np.load("/kaggle/working/preprocessed_t2_val/t2_val_mfcc.npy"),
        np.load("/kaggle/working/preprocessed_t2_val/t2_val_lfcc.npy"),
        np.load("/kaggle/working/preprocessed_t2_val/t2_val_chroma.npy"),
        np.load("/kaggle/working/preprocessed_t2_val/t2_val_labels.npy")
    )
except FileNotFoundError as e:
    raise FileNotFoundError(f"Preprocessed train/val data not found. Run the preprocessing cell first: {e}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
print(f"Train dataset size: {len(train_dataset)}, Val dataset size: {len(val_dataset)}")

# Compute class weights for imbalanced data
train_labels = np.load("/kaggle/working/preprocessed_t2_train/t2_train_labels.npy")
class_counts = np.bincount(train_labels)
class_weights = torch.FloatTensor([1.0 / class_counts[i] for i in range(2)]).to(device)
class_weights = class_weights / class_weights.sum() * 2  # Normalize
print(f"Class weights (Real, Fake): {class_weights.tolist()}")

model = MFAAN().to(device)
try:
    model.load_state_dict(torch.load("/kaggle/working/mfaan_best.pth"))  # Load pre-trained weights
except FileNotFoundError:
    print("Pre-trained model not found. Starting fine-tuning from scratch.")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Low lr for fine-tuning
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Fine-tuning loop
num_epochs = 20
best_val_eer = float("inf")
best_model_path = "/kaggle/working/mfaan_t2_finetuned.pth"

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_preds = []
    train_labels = []
    for mfcc, lfcc, chroma, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(mfcc, lfcc, chroma)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_preds.extend(predicted.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(train_labels, train_preds)
    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    val_preds = []
    val_scores = []
    val_labels = []
    with torch.no_grad():
        for mfcc, lfcc, chroma, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
            mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
            outputs = model(mfcc, lfcc, chroma)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            probs = F.softmax(outputs, dim=1)[:, 1]
            _, predicted = torch.max(outputs, 1)
            val_preds.extend(predicted.cpu().numpy())
            val_scores.extend(probs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    val_eer = compute_eer(val_labels, val_scores)
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%, Val EER: {val_eer*100:.2f}%")

    if val_eer < best_val_eer:
        best_val_eer = val_eer
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best model with Val EER: {best_val_eer*100:.2f}%")

print(f"Fine-tuning complete. Best model saved to {best_model_path}")

# Test the fine-tuned model
model.load_state_dict(torch.load(best_model_path))
model.eval()
print("\nLoaded fine-tuned model for testing.")

# Create test dataset and loader
chunk_dir = "/kaggle/working/preprocessed_t2_test_chunks"
total_t2_test_samples = 16800  # Actual samples, excluding padding
test_indices = list(range(total_t2_test_samples))
t2_test_dataset = AudioDataset(chunk_dir, test_indices, dataset_type="test")
t2_test_loader = DataLoader(t2_test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Evaluate on T2 test set
test_preds = []
test_scores = []
test_labels = []
with torch.no_grad():
    for mfcc, lfcc, chroma, labels in tqdm(t2_test_loader, desc="Evaluating T2 Test Set"):
        mfcc, lfcc, chroma, labels = mfcc.to(device), lfcc.to(device), chroma.to(device), labels.to(device)
        outputs = model(mfcc, lfcc, chroma)
        probs = F.softmax(outputs, dim=1)[:, 1]
        _, predicted = torch.max(outputs, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_scores.extend(probs.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_eer = compute_eer(test_labels, test_scores)
cm = confusion_matrix(test_labels, test_preds)
per_class_acc = cm.diagonal() / cm.sum(axis=1)

# Print results
print(f"\nFinal T2 Test Accuracy: {test_acc * 100:.2f}%")
print(f"Final T2 Test EER: {test_eer * 100:.2f}%")
print(f"Per-class accuracy (Real, Fake): {per_class_acc * 100}")

# Compare with baseline
baseline_accuracy = 68.44
baseline_eer = 40.9
print(f"\nBaseline Accuracy (T2): {baseline_accuracy:.2f}%, Baseline EER: {baseline_eer:.2f}%")
print("Model vs. Baseline:")
print(f"Accuracy: {'Outperforms' if test_acc * 100 > baseline_accuracy else 'Does not outperform'} "
      f"(Model: {test_acc * 100:.2f}%, Baseline: {baseline_accuracy:.2f}%)")
print(f"EER: {'Outperforms' if test_eer * 100 < baseline_eer else 'Does not outperform'} "
      f"(Model: {test_eer * 100:.2f}%, Baseline: {baseline_eer:.2f}%)")

Using device: cuda, GPU: Tesla T4

MFAAN Model Size:
Total Parameters: 206,066
Parameter Size: 0.79 MB
Train dataset size: 134400, Val dataset size: 16800
Class weights (Real, Fake): [1.3333333730697632, 0.6666666865348816]


Epoch 1/20 [Val]: 100%|██████████| 525/525 [00:03<00:00, 150.88it/s]


Epoch 1/20:
Train Loss: 0.2703, Train Acc: 88.65%
Val Loss: 0.3135, Val Acc: 86.97%, Val EER: 10.20%
Saved best model with Val EER: 10.20%


Epoch 2/20 [Val]: 100%|██████████| 525/525 [00:03<00:00, 153.03it/s]


Epoch 2/20:
Train Loss: 0.1972, Train Acc: 92.13%
Val Loss: 0.2405, Val Acc: 90.53%, Val EER: 8.34%
Saved best model with Val EER: 8.34%


...

Epoch 19/20:
Train Loss: 0.1348, Train Acc: 94.60%
Val Loss: 0.2188, Val Acc: 91.32%, Val EER: 7.52%


Epoch 20/20 [Val]: 100%|██████████| 525/525 [00:03<00:00, 150.69it/s]


Epoch 20/20:
Train Loss: 0.1358, Train Acc: 94.55%
Val Loss: 0.2610, Val Acc: 89.70%, Val EER: 8.45%
Fine-tuning complete. Best model saved to /kaggle/working/mfaan_t2_finetuned.pth

Loaded fine-tuned model for testing.
Found 27 chunk files in /kaggle/working/preprocessed_t2_test_chunks. First few: ['t2_test_mfcc_chunk_0.npy', 't2_test_mfcc_chunk_1.npy', 't2_test_mfcc_chunk_10.npy', 't2_test_mfcc_chunk_11.npy', 't2_test_mfcc_chunk_12.npy']
Dataset (test) initialized with 16800 samples.


Evaluating T2 Test Set: 100%|██████████| 525/525 [00:16<00:00, 31.04it/s]


Final T2 Test Accuracy: 91.76%
Final T2 Test EER: 6.89%
Per-class accuracy (Real, Fake): [95.85714286 89.71428571]

Baseline Accuracy (T2): 68.44%, Baseline EER: 40.90%
Model vs. Baseline:
Accuracy: Outperforms (Model: 91.76%, Baseline: 68.44%)
EER: Outperforms (Model: 6.89%, Baseline: 40.90%)
